In [6]:
import json
import numpy as np
from numpy.linalg import norm

# 计算欧几里得距离的函数
def euclidean_distance(g1, g2):
    return norm(g1 - g2)

# 计算欧几里得相似度的函数
def euclidean_similarity(g1, g2):
    distance = euclidean_distance(g1, g2)
    return distance

# 假设梯度文件命名为 'gradients_epoch_X.json', 其中 X 为 epoch 编号
def read_gradients(epoch):
    with open(f'/scratch/liyues_root/liyues/tangzx/mammo/gradient/gradients_epoch_{epoch}.json', 'r') as f:
        gradients_data = json.load(f)
    
    # 提取每个 batch 中的 y1 和 y2 梯度
    y1_gradients = [entry["loss_y1"]["weight_grad"] for entry in gradients_data]
    y2_gradients = [entry["loss_y2"]["weight_grad"] for entry in gradients_data]
    
    return y1_gradients, y2_gradients

# 计算每个 epoch 内所有 batch 的 y1 和 y2 梯度欧几里得相似度
def calculate_epoch_euclidean_similarity(epoch):
    y1_gradients, y2_gradients = read_gradients(epoch)
    
    # 对每个 batch 计算 y1 和 y2 的欧几里得相似度
    euclidean_similarities = []
    for g1, g2 in zip(y1_gradients, y2_gradients):
        g1 = np.array(g1)  # 将列表转换为 NumPy 数组
        g2 = np.array(g2)  # 将列表转换为 NumPy 数组
        euc_sim = euclidean_similarity(g1, g2)
        euclidean_similarities.append(euc_sim)
    
    # 计算该 epoch 的平均欧几里得相似度
    avg_euc_sim = np.mean(euclidean_similarities)
    
    return avg_euc_sim

for epoch in range(1, 51):
    avg_euc_sim = calculate_epoch_euclidean_similarity(epoch)
    print(f'Average Euclidean Similarity between y1 and y2 gradients in Epoch {epoch}: {avg_euc_sim}')


In [2]:
import json
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 计算余弦相似度的函数
def cosine_similarity(g1, g2):
    return dot(g1, g2) / (norm(g1) * norm(g2))

# 假设梯度文件命名为 'gradients_epoch_X.json', 其中 X 为 epoch 编号
def read_gradients(epoch):
    with open(f'/scratch/liyues_root/liyues/tangzx/mammo/gradient/gradients_epoch_{epoch}.json', 'r') as f:
        gradients_data = json.load(f)
    
    # 提取每个 batch 中的 y1 和 y2 梯度
    y1_gradients = [entry["loss_y1"]["weight_grad"] for entry in gradients_data]
    y2_gradients = [entry["loss_y2"]["weight_grad"] for entry in gradients_data]
    
    return y1_gradients, y2_gradients

# 计算每个 epoch 内所有 batch 的 y1 和 y2 梯度余弦相似度
def calculate_epoch_cosine_similarity(epoch):
    y1_gradients, y2_gradients = read_gradients(epoch)
    
    # 对每个 batch 计算 y1 和 y2 的余弦相似度
    cosine_similarities = []
    for g1, g2 in zip(y1_gradients, y2_gradients):
        g1 = np.array(g1)  # 将列表转换为 NumPy 数组
        g2 = np.array(g2)  # 将列表转换为 NumPy 数组
        cos_sim = cosine_similarity(g1, g2)
        cosine_similarities.append(cos_sim)
    
    # 计算该 epoch 的平均余弦相似度
    avg_cos_sim = np.mean(cosine_similarities)

    return avg_cos_sim

# 计算 1-50 个 epoch 的梯度相似度
for epoch in range(1, 51):
    avg_cos_sim = calculate_epoch_cosine_similarity(epoch)
    print(f'Average Cosine Similarity between y1 and y2 gradients in Epoch {epoch}: {avg_cos_sim}')


In [3]:
import json
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 假设梯度文件命名为 'gradients_epoch_X.json', 其中 X 为 epoch 编号
def read_gradients(epoch):
    with open(f'/scratch/liyues_root/liyues/tangzx/mammo/gradient/gradients_epoch_{epoch}.json', 'r') as f:
        gradients_data = json.load(f)
    
    # 提取每个 batch 中的 y1 和 y2 梯度
    y1_gradients = [entry["loss_y1"]["weight_grad"] for entry in gradients_data]
    y2_gradients = [entry["loss_y2"]["weight_grad"] for entry in gradients_data]
    
    return y1_gradients, y2_gradients

# 读取 1 个 epoch 的所有梯度
y1_gradients, y2_gradients = read_gradients(5)  # 可以替换成你感兴趣的 epoch

# 将梯度转换为 NumPy 数组，并展平为一维向量
y1_gradients = np.array([np.array(g).flatten() for g in y1_gradients])  # 展平每个梯度为一维向量
y2_gradients = np.array([np.array(g).flatten() for g in y2_gradients])  # 展平每个梯度为一维向量

# 合并 y1 和 y2 的梯度，以便一起可视化
all_gradients = np.concatenate([y1_gradients, y2_gradients], axis=0)

# 对所有梯度进行 t-SNE 降维，降到 2D
perplexity_value = 3  # 调整 perplexity 值，默认30
learning_rate_value = 200  # 调整 learning_rate 值，默认200
tsne = TSNE(n_components=2, perplexity=perplexity_value, learning_rate=learning_rate_value, random_state=42)

gradients_tsne = tsne.fit_transform(all_gradients)

# 可视化结果
plt.figure(figsize=(10, 6))
plt.scatter(gradients_tsne[:len(y1_gradients), 0], gradients_tsne[:len(y1_gradients), 1], label='y1 Gradients', alpha=0.7)
plt.scatter(gradients_tsne[len(y1_gradients):, 0], gradients_tsne[len(y1_gradients):, 1], label='y2 Gradients', alpha=0.7)
plt.title('t-SNE Visualization of y1 and y2 Gradients (Flattened)')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.legend()
plt.show()


In [86]:
import json
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 假设梯度文件命名为 'gradients_epoch_X.json', 其中 X 为 epoch 编号
def read_gradients(epoch):
    with open(f'/scratch/liyues_root/liyues/tangzx/mammo/gradient/gradients_epoch_{epoch}.json', 'r') as f:
        gradients_data = json.load(f)
    
    # 提取每个 batch 中的 y1 和 y2 梯度
    y1_gradients = [entry["loss_y1"]["weight_grad"] for entry in gradients_data]
    y2_gradients = [entry["loss_y2"]["weight_grad"] for entry in gradients_data]
    
    return y1_gradients, y2_gradients

# 读取 1 个 epoch 的所有梯度
y1_gradients, y2_gradients = read_gradients(6)  # 可以替换成你感兴趣的 epoch



In [4]:
# 将梯度转换为 NumPy 数组，并展平为一维向量
y1_gradients = np.array([np.array(g).flatten() for g in y1_gradients])  # 展平每个梯度为一维向量
y2_gradients = np.array([np.array(g).flatten() for g in y2_gradients])  # 展平每个梯度为一维向量

# 合并 y1 和 y2 的梯度，以便一起可视化
all_gradients = np.concatenate([y1_gradients, y2_gradients], axis=0)

# 对所有梯度进行 t-SNE 降维，降到 2D
perplexity_value = 50  # 调整 perplexity 值，默认30
learning_rate_value = 50  # 调整 learning_rate 值，默认200
tsne = TSNE(n_components=2, perplexity=perplexity_value, learning_rate=learning_rate_value, random_state=42)

gradients_tsne = tsne.fit_transform(all_gradients)

# 可视化结果
plt.figure(figsize=(10, 6))
plt.scatter(gradients_tsne[:len(y1_gradients), 0], gradients_tsne[:len(y1_gradients), 1], label='y1 Gradients', alpha=0.7)
plt.scatter(gradients_tsne[len(y1_gradients):, 0], gradients_tsne[len(y1_gradients):, 1], label='y2 Gradients', alpha=0.7)
plt.title('t-SNE Visualization of y1 and y2 Gradients (Flattened)')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.legend()
plt.show()


In [8]:
def calculate_epoch_euclidean_similarity1(y1_gradients, y2_gradients):
    # = read_gradients(epoch)
    
    # 对每个 batch 计算 y1 和 y2 的欧几里得相似度
    euclidean_similarities = []
    for g1, g2 in zip(y1_gradients, y2_gradients):
        g1 = np.array(g1)  # 将列表转换为 NumPy 数组
        # print(g1.shape)
        g2 = np.array(g2)  # 将列表转换为 NumPy 数组
        euc_sim = cosine_similarity(g1, g2)
        euclidean_similarities.append(euc_sim)
    
    # 计算该 epoch 的平均欧几里得相似度
    print(np.mean(euclidean_similarities))
    print(np.min(euclidean_similarities))
    print(np.max(euclidean_similarities))
    return euclidean_similarities
eu = calculate_epoch_euclidean_similarity1(y1_gradients, y2_gradients)

In [7]:
import numpy as np

# 假设已有的 cosine_similarity 函数
def cosine_similarity(g1, g2):
    return np.dot(g1, g2) / (np.linalg.norm(g1) * np.linalg.norm(g2))

# 计算欧几里得相似度，并按指定区间分类
def calculate_epoch_euclidean_similarity1(y1_gradients, y2_gradients):
    # 对每个 batch 计算 y1 和 y2 的欧几里得相似度
    euclidean_similarities = []
    for g1, g2 in zip(y1_gradients, y2_gradients):
        g1 = np.array(g1)  # 将列表转换为 NumPy 数组
        g2 = np.array(g2)  # 将列表转换为 NumPy 数组
        euc_sim = cosine_similarity(g1, g2)  # 使用余弦相似度代替欧几里得距离
        euclidean_similarities.append(euc_sim)
    
    # 计算该 epoch 的平均、最小、最大欧几里得相似度
    print(f"Mean Similarity: {np.mean(euclidean_similarities)}")
    print(f"Min Similarity: {np.min(euclidean_similarities)}")
    print(f"Max Similarity: {np.max(euclidean_similarities)}")
    
    # 按照 (-1, -0.01), (-0.01, 0.01), (0.01, 1) 分段
    range1 = [sim for sim in euclidean_similarities if -1 <= sim < -0.01]
    range2 = [sim for sim in euclidean_similarities if -0.01 <= sim <= 0.01]
    range3 = [sim for sim in euclidean_similarities if 0.01 < sim <= 1]
    
    # 输出每个区间内的数量
    print(f"(-1, -0.01): {len(range1)} values")
    print(f"(-0.01, 0.01): {len(range2)} values")
    print(f"(0.01, 1): {len(range3)} values")
    
    # 返回分段后的结果
    return {
        "range1 (-1, -0.01)": range1,
        "range2 (-0.01, 0.01)": range2,
        "range3 (0.01, 1)": range3
    }

# 假设 y1_gradients 和 y2_gradients 是从梯度文件中读取的
eu = calculate_epoch_euclidean_similarity1(y1_gradients, y2_gradients)
